Nombre: Felipe Córdova  
Fecha: 09-10-2024

# Tarea 1: Tratamiento Automático del Lenguaje INFO279

Esta tarea incluye dos desafíos que aplican los algoritmos estudiados en clases sobre tratamiento automático del lenguaje.


#### <u>Desafío 1</u>

Construir un modelo de inteligencia artificial capaz de clasificar noticias de prensa según su categoría temática principal. 

Las categorias a detectar con el modelo son las siguientes:  
- ciencia           
- accidentes            
- deportes            
- politica           
- internacional    
- entretenimiento  
- economia        
- medio_ambiente     
- educacion       
- tecnologia   
- salud  

#### <u>Desafío 2</u>

Geolocalizar eventos en noticias utilizando técnicas de tratamiento automático del lenguaje, el objetivo del desafío es estructurar la información de una noticia de la manera siguiente:

- event: principal evento descrito en la noticia
- category: categoria temática del evento
- address: dirección dónde occurió el evento en formato: calle número, comuna, país
- latitud: latitud del evento
- longitud: longitud del evento



#### Evaluación del desafío
Se evaluará su método en base a 100 noticias etiquetadas con las métricas siguientes:

- Category: Precision y Recall
- Longitud y Latitud: Exactitud (con una margen de error de 100 metros)


# Dataset utilizado para los desafios.

Para este problema, se utiliza un dataset llamado [dataset_agosto2024.csv](https://drive.google.com/drive/folders/1YjYSmZmyGIyXYJfoVDy-FNB-NJFPGtFo) obtenido de la organización Sophia2, propiedad del profesor Matthieu Vernier, que realiza scraping de medios de prensa en Chile y recopila todas las noticias del mes de agosto de 2024.

El dataset incluye los siguientes atributos:

- __<u>date</u>__: Fecha de la noticia.
- __<u>media_outlet</u>__: Medio de comunicación donde se publicó la noticia.
- __<u>text</u>__: Contenido de la noticia.
- __<u>url</u>__: Dirección web de la noticia.
- __<u>title</u>__: Título de la noticia.
- __<u>id_news</u>__: Identificador único de la noticia.


Para poder entrenar un modelo de Inteligencia Artificial, se usa el dataset visto en clases de [train_data.csv](https://drive.google.com/drive/folders/1YjYSmZmyGIyXYJfoVDy-FNB-NJFPGtFo). Este dataset es muy similar al anterior, pero en este caso, las noticias ya están etiquetadas. Nuestro objetivo es entrenar un modelo que sea capaz de clasificar correctamente el **dataset_agosto2024**.


1. Lectura dataset de prueba noticias Agosto 2024

In [71]:
import pandas as pd

archivo = "data/dataset_agosto2024.csv"
test_df = pd.read_csv(archivo)

In [72]:
test_df

,date,media_outlet,text,url,title,id_news
0,"Aug 29, 2024 @ 20:00:00.000",cnnchile,"Durante el enfrentamiento, el exfuncionario po...",https://www.cnnchile.com/pais/dos-lesionados-m...,Dos lesionados y un muerto deja enfrentamiento...,53.579.048
1,"Aug 29, 2024 @ 20:00:00.000",elobservador,"LA CALERA.- El jueves 29 de agosto, Carabinero...",https://www.observador.cl/la-calera-con-barric...,La Calera: Con barricada intentaron evitar que...,53.579.054
2,"Aug 29, 2024 @ 20:00:00.000",t13,(empty),https://www.t13.cl/noticia/mundo/juez-del-supr...,Tribunal Supremo de Brasil ordenó bloquear ‘X...,53.578.540
3,"Aug 29, 2024 @ 20:00:00.000",laserenaonline,(empty),https://laserenaonline.cl/2024/08/30/terminal-...,Terminal de buses de Illapel retoma operacione...,53.579.988
4,"Aug 29, 2024 @ 20:00:00.000",laregionhoy,Hasta el viernes 4 de octubre a las 23:59 hora...,https://laregionhoy.cl/2024/08/30/uv-lanza-bas...,UV lanza bases de versión 2024 del Festival Na...,53.580.012
...,...,...,...,...,...,...
20222,"Aug 1, 2024 @ 20:00:00.000",biobiochile,"La ministra del Interior, Carolina Tohá, entre...",https://www.biobiochile.cl/noticias/nacional/r...,Ministra Tohá asegura que se resolvió la emerg...,53.878.844
20223,"Aug 1, 2024 @ 20:00:00.000",biobiochile,"La tarde de ayer jueves, la Brigada de Homicid...",https://www.biobiochile.cl/noticias/nacional/r...,El último viaje de Rosa Lira: las pistas clave...,53.878.403
20224,"Aug 1, 2024 @ 20:00:00.000",biobiochile,Aguas Andinas alerta sobre posibles afectacion...,https://www.biobiochile.cl/noticias/nacional/r...,Aguas Andinas anuncia suspensión de suministro...,53.879.010
20225,"Aug 1, 2024 @ 20:00:00.000",biobiochile,El Gobierno informó la dimisión del subsecreta...,https://www.biobiochile.cl/noticias/nacional/c...,Renuncia Xavier Altamirano a la Subsecretaría ...,53.879.110


Durante el mes de agosto en Chile, se han registrado un total de 20,227 noticias. A continuación, eliminaremos columnas que no sirva, como lo es la fecha, el medio y y el id.

Es importante destacar que algunas filas de la columna 'text' contienen el texto de la noticia vacío (empty), como se observa en las filas 2 y 3. Procederemos a eliminar estas filas, ya que no aportan valor a nuestro desafío de clasificar noticias en las categorías mencionadas.


In [73]:
# Para eliminar múltiples columnas
test_df = test_df.drop(['date', 'media_outlet'], axis=1)

# Elimina filas donde el valor de la columna 'text' es exactamente '(empty)'
test_df = test_df[test_df['text'] != '(empty)']

In [74]:
test_df

,text,url,title,id_news
0,"Durante el enfrentamiento, el exfuncionario po...",https://www.cnnchile.com/pais/dos-lesionados-m...,Dos lesionados y un muerto deja enfrentamiento...,53.579.048
1,"LA CALERA.- El jueves 29 de agosto, Carabinero...",https://www.observador.cl/la-calera-con-barric...,La Calera: Con barricada intentaron evitar que...,53.579.054
4,Hasta el viernes 4 de octubre a las 23:59 hora...,https://laregionhoy.cl/2024/08/30/uv-lanza-bas...,UV lanza bases de versión 2024 del Festival Na...,53.580.012
5,"La jefa comunal, no obstante, valoró el mega o...",https://www.cnnchile.com/pais/claudia-pizarro-...,Alcaldesa Claudia Pizarro asegura que megaoper...,53.579.487
6,La Empresa Nacional de Minería (Enami) anunció...,https://www.emol.com/noticias/Economia/2024/08...,Las gigantes Río Tinto y BYD entre los posible...,53.579.105
...,...,...,...,...
20222,"La ministra del Interior, Carolina Tohá, entre...",https://www.biobiochile.cl/noticias/nacional/r...,Ministra Tohá asegura que se resolvió la emerg...,53.878.844
20223,"La tarde de ayer jueves, la Brigada de Homicid...",https://www.biobiochile.cl/noticias/nacional/r...,El último viaje de Rosa Lira: las pistas clave...,53.878.403
20224,Aguas Andinas alerta sobre posibles afectacion...,https://www.biobiochile.cl/noticias/nacional/r...,Aguas Andinas anuncia suspensión de suministro...,53.879.010
20225,El Gobierno informó la dimisión del subsecreta...,https://www.biobiochile.cl/noticias/nacional/c...,Renuncia Xavier Altamirano a la Subsecretaría ...,53.879.110


Tras limpiar el dataset y eliminar las filas con valores de texto vacíos, ahora contamos con 18,220 noticias. Esto significa que había 2,007 noticias que contenían texto vacío, lo cual no contribuye al análisis. Para verificar esta información, utilizamos el filtro en el buscador de Excel y confirmamos que efectivamente había 2,007 noticias con texto vacío (empty).

<img src="img/img.png" width="600" height="400">



In [75]:
# Restablecer max_colwidth a su valor predeterminado
# pd.set_option('display.max_colwidth', None)
# pd.reset_option('display.max_colwidth')
## Esto es para ver las columnas de df bien

Usarmeos el dataset de **train_data.csv** utilizado en clases, este dataset ya viene con datos etiquetamos, por lo cual podemos usarlo para entrenar al modelo y luego clasificar al dataset de noticias de Agosto.

2. Lectura dataset de entrenamiento

In [76]:
train_df = pd.read_csv('data/train_data.csv')

In [77]:
# Ver todas las etiquetas únicas en la columna 
etiquetas = train_df['clase'].unique()
print(f"Las etiquetas son: {', '.join(etiquetas)}")

Las etiquetas son: politica, tecnologia, accidentes, ciencia, medio_ambiente, salud, entretenimiento, educacion, internacional, economia, deportes


3. Entrenamiento del modelo de Inteligencia Articial

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import spacy

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Definir stop words (puedes personalizar esta lista)
# stop_words = list(nlp.Defaults.stop_words)  # Lista de stop words de spaCy
stop_words = None

# Inicializar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words)

# Crear las matrices de características (X) y etiquetas (y) a partir del dataset de entrenamiento
X = vectorizer.fit_transform(train_df['text'])
y = train_df['clase']

# Dividir el dataset en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

print("Entrenamiento completado.")


Entrenamiento completado.


4. Predecir el modelo ya entrenado

In [79]:
from sklearn.metrics import classification_report

# Predecir las etiquetas para el conjunto de validación
y_pred = model.predict(X_val)

# Mostrar el informe de clasificación para las predicciones en validación
print("Evaluación en el conjunto de validación:")
print(classification_report(y_val, y_pred))

# Preparar el conjunto de test (aplicar el mismo vectorizador a los datos de test)

X_test = vectorizer.transform(test_df['text'])

# Predecir las etiquetas para el conjunto de test
test_df['category'] = model.predict(X_test)

print("Clasificación en el conjunto de test completada...")


Evaluación en el conjunto de validación:
                 precision    recall  f1-score   support

     accidentes       0.73      0.74      0.73       406
        ciencia       0.66      0.43      0.52       414
       deportes       0.53      0.58      0.56       387
       economia       0.58      0.68      0.63       408
      educacion       0.60      0.57      0.59       395
entretenimiento       0.52      0.51      0.51       391
  internacional       0.76      0.87      0.81       380
 medio_ambiente       0.61      0.65      0.63       427
       politica       0.71      0.75      0.73       349
          salud       0.58      0.48      0.53       409
     tecnologia       0.72      0.79      0.75       434

       accuracy                           0.64      4400
      macro avg       0.64      0.64      0.64      4400
   weighted avg       0.64      0.64      0.63      4400

Clasificación en el conjunto de test completada...


Podemos concluir que nuestro modelo de inteligencia artificial ofrece métricas aceptables, con un rendimiento general adecuado.

A continuación, se presentan las principales métricas obtenidas por el modelo:

- accidentes: 73% de precisión y 74% de recall, con un f1-score de 73%.  
- ciencia: La precisión es del 66%, mientras que el recall es del 43%, con un f1-score de 52%.  
- deportes: Muestra una precisión del 53% y un recall del 58%, con un f1-score de 56%.  
- economia: Obtiene una precisión del 58% y un recall del 68%, con un f1-score de 63%.  
- educacion: Presenta una precisión del 60% y un recall del 57%, logrando un f1-score de 59%.  
- entretenimiento: Tiene una precisión del 52% y un recall del 51%, con un f1-score de 51%.  
- internacional: Sobresale con una precisión del 76% y un recall del 87%, alcanzando un f1-score de 81%.  
- medio_ambiente: Registra una precisión del 61% y un recall del 65%, con un f1-score de 63%.  
- politica: Presenta una precisión del 71% y un recall del 75%, con un f1-score de 73%.  
- salud: Muestra una precisión del 58% y un recall del 48%, logrando un f1-score de 53%.  
- tecnologia: Obtiene una precisión del 72% y un recall del 79%, con un f1-score de 75%.  


Se puede ver que las clases con la metrica de recall mas bajas son de 'ciencia' y 'salud'

Aqui estamos guardando todas las noticias clasificadas de acuerdo al modelo.

In [80]:
# Guardar el resultado en un archivo CSV
test_df.to_csv('salidas_intermedias/predicciones_todas.csv', index=False)

5. Guardar en un CSV 100 noticias aleatorias ya clasificadas para el fin del desafio 1.

In [81]:
random_sample = test_df.sample(n=100) # Tomar 100 aleatorias del csv de salida

# Guardar el resultado en un archivo CSV
random_sample.to_csv('salida_desafios/desafio1.csv', index=False) # Esto de acuerdo al enunciado

Ver la cantidad de noticias que fueron clasificadas en cada categoria.

In [82]:
label_counts = random_sample['category'].value_counts()
label_counts

category
accidentes         22
ciencia            16
internacional      12
politica           11
deportes           10
entretenimiento     8
tecnologia          7
economia            5
salud               3
medio_ambiente      3
educacion           3
Name: count, dtype: int64

6. Cargar en un DataFrame el CSV de salida del desafío 1 para utilizarlo en el desafío 2

In [83]:

file = "salida_desafios/desafio1.csv"
salida_df = pd.read_csv(file)

In [84]:
salida_df.head()

,text,url,title,id_news,category
0,Distintos usuarios denunciaron el hecho a trav...,https://www.cnnchile.com/pais/otra-vez-cortes-...,Otra vez cortes de luz en Santiago: Reportan s...,53.427.494,ciencia
1,El centrocampista francés Eduardo Camavinga su...,https://www.biobiochile.cl/noticias/deportes/f...,Real Madrid confirman dura lesión de francés C...,53.877.912,deportes
2,La empresa JetSmart se refirió este lunes a la...,https://www.emol.com/noticias/Nacional/2024/08...,Multa por pasajeros sin documentos: JetSmart d...,53.594.498,internacional
3,Carabineros logró detener a dos sujetos en Cal...,https://www.biobiochile.cl/noticias/nacional/r...,Plan Enjambre de Carabineros permite detener a...,53.565.756,accidentes
4,El carabinero de franco asesinado afuera del T...,https://www.biobiochile.cl/noticias/nacional/r...,Sargento 2do y padre de dos hijos: Rodrigo Pug...,53.878.949,accidentes


Eliminar la columna URL ya que no se utiliza para este desafio

In [85]:
salida_df = salida_df.drop(['url'], axis=1)

In [86]:
salida_df.head()

,text,title,id_news,category
0,Distintos usuarios denunciaron el hecho a trav...,Otra vez cortes de luz en Santiago: Reportan s...,53.427.494,ciencia
1,El centrocampista francés Eduardo Camavinga su...,Real Madrid confirman dura lesión de francés C...,53.877.912,deportes
2,La empresa JetSmart se refirió este lunes a la...,Multa por pasajeros sin documentos: JetSmart d...,53.594.498,internacional
3,Carabineros logró detener a dos sujetos en Cal...,Plan Enjambre de Carabineros permite detener a...,53.565.756,accidentes
4,El carabinero de franco asesinado afuera del T...,Sargento 2do y padre de dos hijos: Rodrigo Pug...,53.878.949,accidentes


In [87]:
# Guardar el resultado en un archivo CSV
# salida_df.to_csv('salidas_intermedias/desafio1_sin_url.csv', index=False)

7. Proceso de Extracción y Geocodificación de Datos de Noticias

In [88]:
import spacy
from geopy.geocoders import Nominatim

# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')

# Inicializar el geolocalizador
geolocator = Nominatim(user_agent="Sophia", timeout=10)


# Función para extraer el evento principal (verbos y sustantivos clave)
def extract_event(text):
    doc = nlp(text)
    event = []
    for token in doc:
        if token.pos_ in ['VERB', 'NOUN']:  # Tomamos los verbos y sustantivos como evento principal
            event.append(token.text)
    return " ".join(event[:5])  # Devolvemos las primeras 5 palabras relevantes

# Función para extraer la dirección utilizando entidades geográficas
def extract_address(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "LOC":  # Si la entidad es una ubicación
            return ent.text
    return None

# Función para geocodificar y obtener latitud y longitud
def geocode_address(address):
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    return None, None

# Procesar cada noticia en el DataFrame
def process_news(df):
    df['event'] = df['text'].apply(extract_event)  # Extraer evento principal
    df['address'] = df['text'].apply(extract_address)  # Extraer dirección
    df['latitud'], df['longitud'] = zip(*df['address'].apply(geocode_address))  # Geocodificar dirección
    return df

# Aplicar el procesamiento a tu dataframe
processed_df = process_news(salida_df)

# Seleccionar las columnas necesarias para la salida
output_df = processed_df[['id_news', 'event', 'category', 'address', 'latitud', 'longitud']]

# Guardar el archivo procesado en formato CSV
output_df.to_csv('salida_desafios/desafio2.csv', index=False)

print("Proceso completo. Archivo guardado como 'salida_resultados.csv'.")


Proceso completo. Archivo guardado como 'salida_resultados.csv'.


Tras revisar el archivo CSV de salida, se puede observar que la mayoría de los datos están correctamente estructurados, especialmente en lo que respecta a la comuna (ciudad), latitud y longitud. Sin embargo, es inevitable que algunos errores persistan, a pesar de los esfuerzos realizados para corregirlos. En relación con la presentación de la columna **address**, no fue posible lograr el formato deseado. Como resultado, se decidió mostrar únicamente la comuna o ciudad donde ocurrió cada noticia. Aunque se intentó incluir la calle y el país utilizando las bibliotecas pertinentes, esto no se logró satisfactoriamente.

Se podría haber utilizado otras técnicas, como CSV adicionales o estructuras de datos que contengan algunas comunas y su respectivo país, pero considero que esto sería una solución "trampa", ya que no se estaría utilizando puramente técnicas de tratamiento automático del lenguaje.